# TCC PUC-Minas Gastão Jucá Filho <br>
Turma Puc-Minas Virtual - nov2019 <br>
Notebook executado <font color=green>no feriado da Pascoa de 2021</font>

#  Atividades: 
> 1. Investigar as 03 (três) tabelas básicas do trabalho: <font color=red>**orders**, **reviews** e **itens**</font> .  
>> 1.1. Checar suas chaves primárias;  
>> 1.2. Analisar deleção de registros; 
> 2. Fazer merges. 
> 3. Documentar tudo com linguagem Markdown.

##  Importação de bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.options.display.max_columns = 100

## Importação das 02 (duas) tabelas

In [2]:
reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep = ',')

In [3]:
orders = pd.read_csv('olist_orders_dataset.csv', sep=',')

## Importação do Diagrama do relacionamento entre as tabelas

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "Order_reviews and Orders_new.bmp")

## Visualização das linhas iniciais das tabelas

In [5]:
reviews.head(3)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24


In [6]:
orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


 ### 1° - Informação sobre a tabela <font color=red>"orders"</font> 

In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
order_id                         99441 non-null object
customer_id                      99441 non-null object
order_status                     99441 non-null object
order_purchase_timestamp         99441 non-null object
order_approved_at                99281 non-null object
order_delivered_carrier_date     97658 non-null object
order_delivered_customer_date    96476 non-null object
order_estimated_delivery_date    99441 non-null object
dtypes: object(8)
memory usage: 6.1+ MB


In [8]:
# ver dimensões do dataset
orders.shape

(99441, 8)

In [9]:
# Qtde de valores únicos de 'order_id'
len(orders['order_id'].unique())

99441

In [10]:
# Qtde de valores únicos de 'customer_id'
len(orders['customer_id'].unique())

99441

In [11]:
# deletar duplicados da concatenação de 'order_id' com 'customer_id': como suas cardinalidades são idênticas e iguais ao
# número de registros da tabela, essa deleção vai atingir 0 (zero) registros, resultando no mesmo número de registros.
len(orders.drop_duplicates (subset=['order_id', 'customer_id']))

99441

#### Portanto:
> O campo 'order_id' **é**  chave primária na tabela <font color=red>"orders"</font> .

#### Adicionalmente:
> - O campo 'customer_id' tem relação de **1-para-1** com a chave primária.  
- Não há registros repetidos.  
- Todos os campos são categóricos.

In [12]:
# vamos repetir a figura das duas tabelas para facilitar referência ao leitor do notebook.
# já resolvemos a tabela da direita, "orders", que tem 99441 registros e o campo 'order_id' como chave primária.
# esse campo é chave estrangeira na tabela que iremos analisar a seguir (tabela "reviews"), a tabela da esquerda.
Image(url= "orders cleaned.bmp")

### 2° - Informação sobre a tabela <font color=red>"reviews"</font>  

In [13]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
review_id                  100000 non-null object
order_id                   100000 non-null object
review_score               100000 non-null int64
review_comment_title       11715 non-null object
review_comment_message     41753 non-null object
review_creation_date       100000 non-null object
review_answer_timestamp    100000 non-null object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [14]:
reviews.shape

(100000, 7)

In [15]:
# Qtde de valores únicos de 'review_id'
len(reviews['review_id'].unique())

99173

In [16]:
# Qtde de valores únicos de 'order_id'
len(reviews['order_id'].unique())

99441

#### Problema: 
> 1. Temos 100.000 registros na tabela <font color=red>"reviews"</font> ;
2. Mas 99.173 valores únicos no campo 'review_id'; e
3. 99.441 valores únicos no campo 'order_id'.

#### Portanto: 
> 1. 'review_id' **não** é a chave primária da tabela.
2. Há mais de uma 'order_id' para uma mesma 'review_id'. E isso é **estranho**.

#### Pergunta:
> 3. Será que há mais de uma 'review_id' para a mesma 'order_id'???

In [17]:
Image(url= "Reviews and Orders being looked into.bmp")

#### Se, dos 100 mil registros, deletarmos os 'review_id' duplicados, vão sobrar...  (a operação não é 'inplace')

In [18]:
len(reviews.drop_duplicates (subset=['review_id']))

99173

#### E, partindo dos mesmos 100 mil, se deletarmos os 'order_id' duplicados, vão sobrar... (not 'inplace either')

In [19]:
len(reviews.drop_duplicates (subset=['order_id']))

99441

#### Ou seja, é uma relação de n-para-n: um 'review_id' tem mais de um 'order_id' e vice-versa.
> É melhor analisar essa relação n x n.

#### <font color=green>Em primeiro lugar, </font> vamos descobrir: dado um 'review_id', quantos 'order_id' há?
> Agrupemos os 'review_id' para obter tal contagem

In [20]:
# o resultado é um Series atribuído à variável 'qtde_order_pra_cada_review'
qtde_order_pra_cada_review = reviews.groupby('review_id')['order_id'].count()

In [21]:
# transformemos esse Series em um dataframe
df_qtde_order_pra_cada_review = pd.DataFrame(qtde_order_pra_cada_review)

In [22]:
# Vamos exibir os 05 (cinco) primeiros registros desse dataframe, após ordenado em ordem decrescente
df_qtde_order_pra_cada_review.sort_values(by='order_id', ascending=False).head()

,order_id
review_id,
dbdf1ea31790c8ecfcc6750525661a9b,3
9e25d6e3025e9b9a0fc7f03588d33e2b,3
2172867fd5b1a55f98fe4608e1547b4b,3
ddc52555ca27b0fe67d5255147682d2d,3
08528f70f579f0c830189efc523d2182,3


In [23]:
# Qual o formato desse dataframe?
df_qtde_order_pra_cada_review.shape

(99173, 1)

In [24]:
# Qtde de 'review_id' que têm 2 ou 3 'order_id'
len(df_qtde_order_pra_cada_review.loc[df_qtde_order_pra_cada_review['order_id'] > 1])

802

In [25]:
# Qtde de 'review_id' que têm 3 'order_id'
len(df_qtde_order_pra_cada_review.loc[df_qtde_order_pra_cada_review['order_id'] > 2])

25

#### Ou seja:  
> há 777 (802 - 25 = 777) *'review_id'* com 2 *'order_id'*  
> Há 25 *'review_id'* com 3 *'order_id'*


#### Vamos dar uma olhada nesses 'review_id' que têm múltiplos 'order_id'

#### Dos 'review_id' que têm 3 'order_id', vamos visualizar na tabela <font color=red>"reviews"</font> aquele que começa com *'dbd'* e termina com *'1a9b'*...

In [26]:
reviews.loc[reviews['review_id'] == 'dbdf1ea31790c8ecfcc6750525661a9b']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
24420,dbdf1ea31790c8ecfcc6750525661a9b,fa06c9f04ef55a2a43f2246a0777b7c8,1,NaN,O produto veio embalado apenas por um frágil e...,2018-03-20 00:00:00,2018-03-21 01:02:05
49929,dbdf1ea31790c8ecfcc6750525661a9b,3cf387bb14e9db171ccbb9b87ea607bb,1,NaN,O produto veio embalado apenas por um frágil e...,2018-03-20 00:00:00,2018-03-21 01:02:05
96293,dbdf1ea31790c8ecfcc6750525661a9b,9406240a4e41945ba492020a2702f757,1,NaN,O produto veio embalado apenas por um frágil e...,2018-03-20 00:00:00,2018-03-21 01:02:05


#### Visualizemos na tabela <font color=red>"reviews"</font> um 2° 'review_id' (o que começa com '9e25' e termina com '3e2b')

In [27]:
reviews.loc[reviews['review_id'] == '9e25d6e3025e9b9a0fc7f03588d33e2b']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
45528,9e25d6e3025e9b9a0fc7f03588d33e2b,869997fbe01f39d184956b5c6bccfdbe,1,Razoavel,Peço um produto por um código e vem outro tota...,2018-08-31 00:00:00,2018-09-03 09:33:00
72401,9e25d6e3025e9b9a0fc7f03588d33e2b,0d3adebce4bebc1f80a7f36e9833f497,1,Razoavel,Peço um produto por um código e vem outro tota...,2018-08-31 00:00:00,2018-09-03 09:33:00
76542,9e25d6e3025e9b9a0fc7f03588d33e2b,52b7fa35b1e5c8bdea5869804dced415,1,Razoavel,Peço um produto por um código e vem outro tota...,2018-08-31 00:00:00,2018-09-03 09:33:00


#### olhemos na tabela <font color=red>"reviews"</font> um 3° 'review_id' (o que começa com '2172' e termina com '7b4b')

In [28]:
reviews.loc[reviews['review_id'] == '2172867fd5b1a55f98fe4608e1547b4b']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
15243,2172867fd5b1a55f98fe4608e1547b4b,559d606ac642899e44550f194fec7e08,5,NaN,Entrega no prazo e produto de qualidade!,2018-02-15 00:00:00,2018-02-26 15:53:18
28806,2172867fd5b1a55f98fe4608e1547b4b,ac6e61336e852cdc45fe59ada3763a66,5,NaN,Entrega no prazo e produto de qualidade!,2018-02-15 00:00:00,2018-02-26 15:53:18
54384,2172867fd5b1a55f98fe4608e1547b4b,e11ba7fd8fe0728dcd89efddcda9fb11,5,NaN,Entrega no prazo e produto de qualidade!,2018-02-15 00:00:00,2018-02-26 15:53:18


#### Mesmo efeito nos três casos: um mesmo 'review_id' refere-se a 03 (três) pedidos ('order_id'), sendo que os atributos são <font color=green>idênticos</font>. É um defeito da tabela <font color=red>"reviews"</font> causado por falha de carregamento dos dados.

In [29]:
# Repetição da figura pra facilitar leitor acompanhar a análise
Image(url= "Reviews and Orders being looked into.bmp")

#### O que fazer????  
> Simples: deletar as redundâncias de 'review_id'!

In [30]:
review_id_sem_repetir = reviews.drop_duplicates (subset=['review_id']) # o default é manter a primeira ocorrência da duplicação

In [31]:
review_id_sem_repetir.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99173 entries, 0 to 99999
Data columns (total 7 columns):
review_id                  99173 non-null object
order_id                   99173 non-null object
review_score               99173 non-null int64
review_comment_title       11665 non-null object
review_comment_message     41431 non-null object
review_creation_date       99173 non-null object
review_answer_timestamp    99173 non-null object
dtypes: int64(1), object(6)
memory usage: 6.1+ MB


In [32]:
review_id_sem_repetir.shape

(99173, 7)

In [33]:
len(review_id_sem_repetir['review_id'].unique())

99173

In [34]:
len(review_id_sem_repetir['order_id'].unique())

98926

In [35]:
# Nova configuração de tabelas
Image(url= "review_id_sem_repetir e orders.bmp")

#### A relação entre 'review_id' e 'order_id' (tabela da esquerda na figura acima) ainda não é 1-pra-1.
> Pois existe diferença entre os valores únicos (99.173 - 98.926 = 247)  
>Essa diferença precisa, *necessariamente*, ser eliminada????? É possível existir um review para mais de uma order?????
>> vamos terminar a análise sobre 'review_id' x 'order_id' a seguir.

#### <font color=green>Em primeiro lugar, </font> (mais acima) descobrimos quantos 'order_id' há em um dado 'review_id'.
> Isso resultou na deleção das redundâncias de 'review_id' e a criação da tabela <font color=red>"review_id_sem_repetir", </font>

#### <font color=green>Em segundo lugar, </font> (agora) vamos responder à pergunta inversa: 
> Quantos 'review_id'  há em um dado 'order_id'?
>> Agrupemos os 'order_id' para obter a contagem.

> Faremos isso na nova tabela <font color=red>"review_id_sem_repetir"

In [36]:
# o resultado é um Series atribuído à variável 'qtde_review_pra_cada_order'
qtde_review_pra_cada_order = review_id_sem_repetir.groupby('order_id')['review_id'].count()

In [37]:
# transformemos 'qtde_review_pra_cada_order' em um dataframe
df_qtde_review_pra_cada_order = pd.DataFrame(qtde_review_pra_cada_order)

In [38]:
# Vamos exibir os 05 (cinco) primeiros registros desse dataframe, após ordenado em ordem decrescente de 'review_id'
df_qtde_review_pra_cada_order.sort_values(by='review_id', ascending=False).head()

,review_id
order_id,
7845a2492ab1b4f2cf3d56c7b8da1446,2
8ff88873f03f1912c00741f8e5ae6c79,2
23a2d8b0357730d4f9c50aa48773e9e9,2
9de38c42ea4d33af19aac6c4d6c4b146,2
2792c4f95b24c1a72641cd404561bdd8,2


In [39]:
df_qtde_review_pra_cada_order.shape

(98926, 1)

In [40]:
# Qtde de 'orders' que têm 2 ou 3 'reviews'
len(df_qtde_review_pra_cada_order.loc[df_qtde_review_pra_cada_order['review_id'] > 1])

247

In [41]:
# Qtde de 'orders' que têm 3 'reviews'
len(df_qtde_review_pra_cada_order.loc[df_qtde_review_pra_cada_order['review_id'] > 2])

0

##### Ou seja:  
> há 247 *'order_id'* com 2 *'review_id'*  

> Não há *'order_id'* com 3 *'review_id'*


#### Dos 'order_id' que têm 2 *'review_id'*, olhemos na tabela <font color=red>"review_id_sem_repetir"</font> o que começa com *'784'* e termina com *'446'*...

In [42]:
review_id_sem_repetir.loc[review_id_sem_repetir['order_id'] == '7845a2492ab1b4f2cf3d56c7b8da1446']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
57360,ae501aff8d8cc761c92008e6afd9dcaa,7845a2492ab1b4f2cf3d56c7b8da1446,5,NaN,chegou antesdo programado,2018-01-24 00:00:00,2018-01-30 19:19:56
73064,8566d94f7bf3bedbb4ca6a29725fc382,7845a2492ab1b4f2cf3d56c7b8da1446,5,NaN,chegou antes do prometido,2018-01-27 00:00:00,2018-01-30 19:18:27


#### Olhemos na tabela <font color=red>"review_id_sem_repetir"</font> um 2° 'order_id' (o que começa com '8ff' e termina com 'c79')

In [43]:
review_id_sem_repetir.loc[review_id_sem_repetir['order_id'] == '8ff88873f03f1912c00741f8e5ae6c79']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
12028,51863339d4ef48a47f43da8b1de705ff,8ff88873f03f1912c00741f8e5ae6c79,1,NaN,NaN,2018-02-21 00:00:00,2018-02-24 10:07:29
16041,d5bf42808fd0df2b766d5c8ece19b3b6,8ff88873f03f1912c00741f8e5ae6c79,5,NaN,NaN,2018-02-01 00:00:00,2018-02-05 18:51:59


#### Olhemos na tabela <font color=red>"review_id_sem_repetir"</font> um 3° 'order_id' (o que começa com '23a' e termina com '9e9')

In [44]:
review_id_sem_repetir.loc[review_id_sem_repetir['order_id'] == '23a2d8b0357730d4f9c50aa48773e9e9']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
8915,873a5bc219517a3b5cde7782127cd823,23a2d8b0357730d4f9c50aa48773e9e9,5,NaN,NaN,2018-04-06 00:00:00,2018-04-07 11:25:14
40703,29d02d3bf25077ca7e448c746bc85ec4,23a2d8b0357730d4f9c50aa48773e9e9,5,NaN,NaN,2018-03-22 00:00:00,2018-03-23 01:26:53


#### Mesmo efeito nos três casos (um 'order_id' para dois 'review_id'). Não é necessariamente incongruência. Isso pode significar, por exemplo, que foram emitidos reviews para itens diferentes de um mesmo pedido. 
> Vamos ter de confirmar isso, checando o número de itens desses pedidos na tabela <font color=red>"itens"</font> (olist_order_items_dataset).

## Importação da tabela <font color=red>"itens"</font>

In [45]:
itens = pd.read_csv("olist_order_items_dataset.csv", sep = ',')

## Importação do Diagrama do relacionamento entre as tabelas <font color=red>"orders"</font> e <font color=red>"itens"</font>.

In [46]:
Image(url= "Order_itens and Orders.bmp")

In [47]:
itens.shape

(112650, 7)

In [48]:
itens.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


#### 'order_id' com 2 reviews, obtido da tabela <font color=red>"review_id_sem_repetir"</font>. 
>Vamos checá-lo na tabela "itens"

In [49]:
itens.loc[itens['order_id'] == '7845a2492ab1b4f2cf3d56c7b8da1446']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
52849,7845a2492ab1b4f2cf3d56c7b8da1446,1,16241e79b047426033194065d8b321b2,4830e40640734fc1c52cd21127c341d4,2018-01-24 12:56:40,29.99,14.10


#### Só há 1 item. Então as duas instâncias na tabela "reviews" eram incongruências.  
> Vamos testar outro 'order_id'

In [50]:
# Outro 'order_id' com 2 reviews, obtido da tabela "review_id_sem_repetir". Selecionando-o na tabela "itens"
itens.loc[itens['order_id'] == '8ff88873f03f1912c00741f8e5ae6c79']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
63069,8ff88873f03f1912c00741f8e5ae6c79,1,75d6b6963340c6063f7f4cfcccfe6a30,cc419e0650a3c5ba77189a1882b7556a,2018-01-31 02:53:02,56.99,14.15


#### Mais uma vez, só há 1 item. Estranho.
> Vamos ainda testar mais outro 'order_id'.

In [51]:
# E mais outro 'order_id' com 2 reviews, obtido da tabela "review_id_sem_repetir".
itens.loc[itens['order_id'] == '23a2d8b0357730d4f9c50aa48773e9e9']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
15598,23a2d8b0357730d4f9c50aa48773e9e9,1,b43ceb87eafb3eb7ceab88d9dd8645af,83f2b4881141ce3fcd4e985125d3279e,2018-03-23 15:08:04,34.90,12.79


#### Terceira vez em que só há 1 item. Amostragem desagradável que sugere o seguinte:
> Era infundada a hipótese de que os 02 (dois) 'review_id' para a mesma 'order_id' (na tabela "review_id_sem_repetir") deviam-se a vários itens na mesma 'order_id'.

> O que fazer com esta informação?
>> Resposta: deletar os 'order_id' duplicados na tabela "review_id_sem_repetir".

In [52]:
Image(url= "review_id_sem_repetir e orders.bmp")

In [53]:
reviews_normalizada = review_id_sem_repetir.drop_duplicates (subset=['order_id']) # Será mantida a 1ª ocorrência.

In [54]:
reviews_normalizada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98926 entries, 0 to 99999
Data columns (total 7 columns):
review_id                  98926 non-null object
order_id                   98926 non-null object
review_score               98926 non-null int64
review_comment_title       11659 non-null object
review_comment_message     41343 non-null object
review_creation_date       98926 non-null object
review_answer_timestamp    98926 non-null object
dtypes: int64(1), object(6)
memory usage: 6.0+ MB


In [55]:
reviews_normalizada.shape

(98926, 7)

In [56]:
# Qtde de valores únicos de 'review_id'
len(reviews_normalizada['review_id'].unique())

98926

In [57]:
# Qtde de valores únicos de 'order_id'
len(reviews_normalizada['order_id'].unique())

98926

In [58]:
# Agora, com a supressão de duplicatas, tanto de 'review_id', quanto de 'order_id', a tabela "reviews" ficou normalizada.
Image(url= "reviews_normalizada e orders.bmp")

#### Agora:
> 'review_id' é a chave primária da tabela "reviews_normalizada" (tabela da esquerda); e

> 'order_id' é a chave primária da tabela "orders" (da direita).

> Nem todas os pedidos recebem reviews 
>> n° de 'review_id' (de <font color=red>"reviews_normalizada"</font>) é menor que o n° de 'order_id' (de <font color=red>"orders"</font>)

In [59]:
# Número de orders que não têm reviews
print(99441-98926)

515


### Análise das 03 (três) tabelas conjuntamente

#### Antes, porém, vamos complementar algumas informações da tabela "itens"

In [60]:
# Qtde de valores únicos de 'order_id'
len(itens['order_id'].unique())

98666

In [61]:
# Qtde de valores únicos de 'order_item_id'
len(itens['order_item_id'].unique())

21

In [62]:
type(itens['order_item_id'])

pandas.core.series.Series

In [63]:
# Qtde de valores únicos de 'product_id'
len(itens['product_id'].unique())

32951

In [64]:
# Qtde de valores únicos de 'seller_id'
len(itens['seller_id'].unique())

3095

In [65]:
Image(url= "Orders e Itens.bmp")

#### Até agora estamos trabalhamos com as três tabelas abaixo

In [66]:
Image (url = "tres tabelas.bmp")

#### Exploremos mais um pouco essa tabela <font color=red>"itens"</font>...

In [67]:
# Formato
itens.shape

(112650, 7)

In [68]:
# Qual a chave primária dela?
# Vamos deletar duplicados da concatenação de 'order_id' com 'order_item_id' (Se não houver deleção, eis a chave primária)
len(itens.drop_duplicates (subset=['order_id', 'order_item_id']))

112650

##### Bingo: a chave primária de <font color=red>"itens"</font> é a concatenação dos campos 'order_id' e 'order_item_id'

In [69]:
# valores únicos
len(itens['order_id'].unique())

98666

In [70]:
# valores únicos
len(itens['order_item_id'].unique())

21

In [71]:
# valores únicos de modo mais simples
itens['order_id'].nunique()

98666

In [72]:
# valores únicos de modo mais simples
itens['order_item_id'].nunique()

21

In [73]:
# Como só há 21 (vinte e um) valores  de 'order_item_id', vamos agrupá-lo e contar 'order_id'
Order_item_id_agrupados = itens.groupby('order_item_id')['order_id'].count()

# vamos transformar em um Dataframe o Series resultante desse GroupBy 
df_Order_item_id_agrupados = pd.DataFrame(Order_item_id_agrupados)

# Visualizemos qtos 'order_id' há em cada 'ordem_item_id'
df_Order_item_id_agrupados

,order_id
order_item_id,
1,98666
2,9803
3,2287
4,965
5,460
6,256
7,58
8,36
9,28


#### Uáu! 
> 98.666 pedidos têm 1 item.  
> Nos restantes, os clientes pediram mais de um produto.
>> Por exemplo:
>>> 17 pedidos ('order_id') têm 11 itens ;  
>>> 5 pedidos têm 15 itens; (subentende-de 1 produto em cada item)    
>>> 3 pedidos têm 19 itens;  
>>> <font color=green>e somente 1 pedido têm 21 itens</font>    
(caramba, esse cliente pediu muita coisa, hein? Que tal olhar esse pedido?

In [74]:
# Qtde de 'order_id' com mais de 19 itens.
len(itens.loc[itens['order_item_id'] > 19])

4

In [75]:
# Listagem dos 'order_id' com mais de 19 itens.
itens.loc[itens['order_item_id'] > 19]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
11951,1b15974a0141d54e36626dca3fdc731a,20,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
57316,8272b63d03f5f79c56e9e4120aec44ef,20,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57317,8272b63d03f5f79c56e9e4120aec44ef,21,79ce45dbc2ea29b22b5a261bbb7b7ee7,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,7.80,6.57
75122,ab14fdcfbe524636d65ee38360e22ce8,20,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44


In [76]:
# Listagem do 'order_id' que tem 21 itens (obtido acima)
itens.loc[itens['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89
57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.20,7.89


#### Análise das informações obtidas acima: 
> 1. Os registros estão em sequência (index 57297 a 57317);
2. O campo 'order_id' tem sempre o mesmo valor;
3. O campo 'order_item_id' vai de 1 a 21 (já que pedimos exatamento isso);
4. O campo 'product_id' DEVERIA ter valores diferentes...
>> Mas
>>> Têm o mesmo valor os itens 1 e 12 a 20;   
Têm o mesmo valor, e distinto do anterior, os itens 2 a 11.    
O item 21 tem um 3° valor distinto.    


> 5. O campo 'seller_id' tem o mesmo valor
>> É normal.
6. O campo 'shipping_limit_date' tem o mesmo valor
>> É normal, já que o vendedor é o mesmo para os 21 itens do pedido.
7. O campo 'price' tem o mesmo valor, exceto pelo item 21. 
8. O campo 'freight_value' tem o mesmo valor, exceto pelo item 21.

#### Portanto: 
> 1. O item 4 acima <font color=red>NÃO</font> é normal!
>> Onde já se viu o cliente pedir várias vezes o mesmo produto em um mesmo pedido?
2. Essa é mais uma falha de carregamento do banco de dados.

#### Decisão:
> Vamos investigar mais um 'order_id' de múltiplos itens.

In [77]:
# Listagem do 'order_id' que tem 20 itens (obtido acima)
itens.loc[itens['order_id'] == 'ab14fdcfbe524636d65ee38360e22ce8']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
75103,ab14fdcfbe524636d65ee38360e22ce8,1,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75104,ab14fdcfbe524636d65ee38360e22ce8,2,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75105,ab14fdcfbe524636d65ee38360e22ce8,3,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75106,ab14fdcfbe524636d65ee38360e22ce8,4,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75107,ab14fdcfbe524636d65ee38360e22ce8,5,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75108,ab14fdcfbe524636d65ee38360e22ce8,6,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75109,ab14fdcfbe524636d65ee38360e22ce8,7,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75110,ab14fdcfbe524636d65ee38360e22ce8,8,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75111,ab14fdcfbe524636d65ee38360e22ce8,9,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44
75112,ab14fdcfbe524636d65ee38360e22ce8,10,9571759451b1d780ee7c15012ea109d4,ce27a3cc3c8cc1ea79d11e561e9bebb6,2017-08-30 14:30:23,98.70,14.44


#### 20 'product_id' iguais???? Come on, man, that's impossible in a real world...
> Isso é falha de carregamento dos dados;  

> Alguém aí tem alguma outra explicação??????


In [78]:
# Só mais um:'order_id' que tem 20 itens
itens.loc[itens['order_id'] == '1b15974a0141d54e36626dca3fdc731a']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
11932,1b15974a0141d54e36626dca3fdc731a,1,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11933,1b15974a0141d54e36626dca3fdc731a,2,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11934,1b15974a0141d54e36626dca3fdc731a,3,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11935,1b15974a0141d54e36626dca3fdc731a,4,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11936,1b15974a0141d54e36626dca3fdc731a,5,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11937,1b15974a0141d54e36626dca3fdc731a,6,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11938,1b15974a0141d54e36626dca3fdc731a,7,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11939,1b15974a0141d54e36626dca3fdc731a,8,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11940,1b15974a0141d54e36626dca3fdc731a,9,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12
11941,1b15974a0141d54e36626dca3fdc731a,10,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-01 02:50:48,100.00,10.12


#### De novo, 20 'product_id' iguais.
> tsc tsc tsc

### Vamos deletar essas redundâncias: todos os 'order_id' ficarão com somente 1 'order_item_id'

#### Mas, antes, por pura curiosidade, olhemos na tabela <font color=red>"produtos"</font>  quais são esses 'product_id' repetidos

In [79]:
produtos = pd.read_csv("olist_products_dataset.csv", sep = ',')

In [80]:
# Selecionando um 'product_id' na tabela "produtos"
produtos.loc[produtos['product_id'] == 'ee3d532c8a438679776d222e997606b3']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
27921,ee3d532c8a438679776d222e997606b3,informatica_acessorios,43.00,452.00,1.00,360.00,19.00,18.00,15.00


In [81]:
# Selecionando outro
produtos.loc[produtos['product_id'] == '9571759451b1d780ee7c15012ea109d4']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
25654,9571759451b1d780ee7c15012ea109d4,automotivo,39.00,"1,042.00",1.00,"1,400.00",20.00,31.00,30.00


In [82]:
# E mais outro
produtos.loc[produtos['product_id'] == '270516a3f41dc035aa87d220228f844c']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
2742,270516a3f41dc035aa87d220228f844c,beleza_saude,45.00,232.00,3.00,800.00,21.00,4.00,15.00


In [83]:
# Um último...
produtos.loc[produtos['product_id'] == '05b515fdc76e888aada3c6d66c201dff']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
16218,05b515fdc76e888aada3c6d66c201dff,beleza_saude,45.00,231.00,3.00,800.00,21.00,4.00,15.00


In [84]:
produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
product_id                    32951 non-null object
product_category_name         32341 non-null object
product_name_lenght           32341 non-null float64
product_description_lenght    32341 non-null float64
product_photos_qty            32341 non-null float64
product_weight_g              32949 non-null float64
product_length_cm             32949 non-null float64
product_height_cm             32949 non-null float64
product_width_cm              32949 non-null float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [85]:
len(produtos['product_id'].unique())

32951

In [86]:
produtos['product_id'].nunique()

32951

In [87]:
len(produtos['product_category_name'].unique())

74

In [88]:
produtos['product_category_name'].nunique()

73

#### Não há a descrição do nome do produto, somente o seu código. Não podemos ver qual produto é, apenas a sua categoria. Há 74 categorias para os mais de 32 mil produtos.
> 'product_id' é a chave primária da tabela <font color=red>"produtos"</font>

#### Deleção das redundâncias da tabela <font color=red>"itens"</font> (tabela da direita)

In [89]:
# situação ANTES da deleção
Image (url = "tres tabelas.bmp")

In [90]:
itens_sem_repetir = itens.drop_duplicates (subset=['order_id']) # Será mantida a 1ª ocorrência.

In [91]:
type(itens_sem_repetir)

pandas.core.frame.DataFrame

In [92]:
itens_sem_repetir.shape

(98666, 7)

In [93]:
itens_sem_repetir['order_id'].nunique()

98666

In [94]:
itens_sem_repetir['order_item_id'].nunique()

1

In [95]:
itens_sem_repetir['product_id'].nunique()

31881

In [96]:
itens_sem_repetir['seller_id'].nunique()

3088

In [97]:
itens['product_id'].nunique()

32951

In [98]:
# situação APÓS da deleção
Image (url = "tres tabelas pos delecao.bmp")

#### Apesar de a nova tabela <font color=red>"itens_sem_repetir"</font> falhar em informar sobre itens, ainda é útil para informar qual o produto e qual o fornecedor estão atrelados a cada pedido.
> No entanto, vamos desprezar em uma futura operação de 'Join' as colunas restantes, pois:  
>> Os preços do produto e do frete influenciariam somente em uma análise de 'Churn' que não é o nosso caso.  
>> A data de envio também é desnecessária, uma vez que temos datas diversas na tabela <font color=red>"orders"</font>.